# 券商分點

這個券商分點可以繞過「圖片認證碼」請大家再三保密使用XD
不然哪天就失效了，

抓取券商分點，每檔股票總共可以分成三個步驟：
1. 開啟驗證碼網頁，得到驗證碼
2. 利用驗證碼來得到券商分點
3. 整理券商分點資訊

In [1]:
# ---------------
# 偽裝成瀏覽器
# ---------------
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
    '1 Firefox/4.0.1',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'en-us,en;q=0.5',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}

# 1. 開啟驗證碼網頁，得到驗證碼
這邊的驗證碼不是「圖片驗證碼喔！」而是 `viewstate` 跟 `eventvalidation`

In [2]:
# ---------------
# 開啟有驗證碼的網頁
# ---------------
import requests
rs = requests.session()

def get_verify_code(rs):
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsMenu.aspx', stream = True, verify = False, headers = headers, timeout=None )
    # ---------------
    # 從網頁中抓取一些關鍵字
    # ---------------
    import re
    
    # get view state
    viewstate = re.search( 'VIEWSTATE"\s+value=.*=', res.text )
    viewstate = viewstate.group()[18:]
    
    # get eventvalidation
    eventvalidation = re.search( 'EVENTVALIDATION"\s+value=.*\w', res.text )
    eventvalidation = eventvalidation.group( )[ 24: ]
    
    return viewstate, eventvalidation

viewstate, eventvalidation = get_verify_code(rs)

print('----')
print('viewstate', viewstate)
print('----')
print('eventvalidation',eventvalidation)
print('----')

C:\Users\Dandy\Anaconda3\envs\finlab\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


----
viewstate /wEPDwUJOTAxMjkxMjg3D2QWAgIDD2QWBAIBD2QWAmYPZBYEZg9kFgJmDxAPFgYeB0NoZWNrZWRnHglGb3JlQ29sb3IKAB4EXyFTQgIEZGRkZAIBD2QWAgIBDw8WBB8BCgAfAgIEZGQCAw9kFgQCAw8PFgIeBFRleHRlZGQCCQ8PFgIeB1Zpc2libGVoZGQYAgUeX19Db250cm9sc1JlcXVpcmVQb3N0QmFja0tleV9fFgMFElJhZGlvQnV0dG9uX05vcm1hbAUQUmFkaW9CdXR0b25fRXhjZAUQUmFkaW9CdXR0b25fRXhjZAUPQ2FwdGNoYUNvbnRyb2wxDwUkMmI4NTExMWYtMmE2MC00YWE3LTgyZWUtNTk4OTAyZWQ4YmMxZMKrCJH26duPypQQ+05vLd4=
----
eventvalidation /wEdAAacPGDkZvixf0XsbXEcl/OQfMmuxAJNAJcNkRRsVeJwwqKURZav/+YrVMqaWE2hvMxJwK4Ohf3nRgXHkFhQen1PRSciNdPYWmINCrip1wqw01PJCA0uL9aE2sjICZqv6GrQAmrXGlXLo8lSiiAsmKsB
----


In [ ]:
ve

# 2. 透過驗證碼爬取資料並存檔

In [4]:
# ---------------
# 爬取資料
# ---------------

def get_data(rs, stock_id, viewstate, eventvalidation):
    import time
    payload = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__LASTFOCUS':'',
    '__VIEWSTATE' : viewstate,                      #encode_viewstate[:-1],
    '__EVENTVALIDATION' : eventvalidation,          #encode_eventvalidation[:-1],
    'RadioButton_Normal' : 'RadioButton_Normal',
    'TextBox_Stkno' : stock_id,
    'CaptchaControl1 ' : 'Z67YB',
    'btnOK' : '%E6%9F%A5%E8%A9%A2',
    }
    rs.post( "https://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload, headers=headers, stream = True )
    time.sleep(1)
    res = rs.get( 'https://bsr.twse.com.tw/bshtm/bsContent.aspx',verify = False ,stream = True, )
    return res

res = get_data(rs, '1101', viewstate, eventvalidation)
res.encoding='big5'
open('test.html', 'w', encoding='utf-8').write(res.text)

C:\Users\Dandy\Anaconda3\envs\finlab\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\Dandy\Anaconda3\envs\finlab\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


50525

# 3. 整理資料

In [5]:
import pandas as pd
from io import StringIO

def parse_data(text):
    lines = text.split('\n')
    lines = [l for l in lines if len(l.split(',')) == 11]
    df = pd.read_csv(StringIO('\n'.join(lines)))

    first_df = df[df.columns[:5]]
    second_df = df[df.columns[6:]]
    second_df.columns = second_df.columns.str.replace('.1', '')
    final_df = first_df.append(second_df).set_index('序號').sort_index().dropna()
    return final_df




df = parse_data(res.text)

# 完整下載範例（每天來按一下）

假如無法下載，可以參考
http://finlabcourse.imotor.com/viewthread.php?tid=322&extra=page%3D1&page=3
來做一些更動，就可以順利執行了～

In [7]:
import time
import requests
from io import StringIO
import pandas as pd
import datetime

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
    '1 Firefox/4.0.1',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'en-us,en;q=0.5',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}

def get_data(rs, stock_id, viewstate, eventvalidation):
    import time
    payload = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__LASTFOCUS':'',
    '__VIEWSTATE' : viewstate,                      #encode_viewstate[:-1],
    '__EVENTVALIDATION' : eventvalidation,          #encode_eventvalidation[:-1],
    'RadioButton_Normal' : 'RadioButton_Normal',
    'TextBox_Stkno' : stock_id,
    'CaptchaControl1 ' : 'Z67YB',
    'btnOK' : '%E6%9F%A5%E8%A9%A2',
    }

    rs.post( "https://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload, headers=headers)#, verify = False, stream = True )
    time.sleep(1)
    res = rs.get( 'https://bsr.twse.com.tw/bshtm/bsContent.aspx')#,verify = False ,stream = True, )
    return res

def get_verify_code(rs):
    res = rs.get( 'https://bsr.twse.com.tw/bshtm/bsMenu.aspx', stream = True, verify = False, headers = headers, timeout=None )
    # ---------------
    # 從網頁中抓取一些關鍵字
    # ---------------
    import re
    
    # get view state
    viewstate = re.search( 'VIEWSTATE"\s+value=.*=', res.text )
    viewstate = viewstate.group()[18:]
    
    # get eventvalidation
    eventvalidation = re.search( 'EVENTVALIDATION"\s+value=.*\w', res.text )
    eventvalidation = eventvalidation.group( )[ 24: ]
    
    return viewstate, eventvalidation

def download_stock(stock_id, path):
    rs = requests.Session()
    #viewstate, eventvalidation = get_verify_code(rs)
    
    while True:
        try:
            viewstate, eventvalidation = get_verify_code(rs)
            break # 抓資料成功,進行下一步
        except Exception as e:
            print(e) # 印出無法連線原因
            print('無法拿到資料, 等 31 sec') # server好像會擋30sec
            time.sleep(31)
            continue # 抓資料失敗,重新抓資料
    
    time.sleep(2)
    res = get_data(rs, stock_id, viewstate, eventvalidation)
    res.encoding='big5'
    open(path, 'w', encoding='utf-8').writelines(res.text)
    
def stock_list():
    res = requests.get('https://dts.twse.com.tw/opendata/t187ap03_L.csv')
    res.encoding='utf-8'
    df = pd.read_csv(StringIO(res.text), index_col=['公司代號'])
    return df

import os

# build main dir
main_dir = os.path.join('data', 'broker_history')
if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
    
# build date dir
now = datetime.datetime.now()
date_dir = os.path.join(main_dir, now.strftime('%Y%m%d'))
if not os.path.isdir(date_dir):
    os.mkdir(date_dir)

    
slist = stock_list()
for s in slist.index:
    stock_path = os.path.join(date_dir, str(s) + '.csv')
    print(stock_path)
    if not os.path.isfile(stock_path):
        print('downloading...')
        download_stock(str(s), stock_path)
        time.sleep(10)
    

FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: 'data\\broker_history'

# 券商總買賣

In [8]:
buy = df['買進股數'].astype(int).groupby(df['券商']).sum()
sell= df['賣出股數'].astype(int).groupby(df['券商']).sum()
total = buy - sell
total

券商
1020合　　庫       59000
1021合庫台中        5000
1022合庫台南        3000
1023合庫高雄        4000
1024合庫嘉義        4000
1025合庫基隆        7000
1028合庫彰化        2000
102A合庫新竹       -2300
102C合庫自強      -25003
102E合庫桃園        1000
102F合庫西台        1000
1030土　　銀        1000
1031土銀台中        5000
1032土銀台南        6000
1033土銀高雄        2000
1035土銀新竹       14000
1037土銀花蓮        2000
1038土銀和平           0
1039土銀士林        1000
103A土銀建國        4000
103C土銀白河        3000
103F土銀南港       10000
1040臺    銀     -4000
1041臺銀鳳山        4000
1042臺銀臺南       -4000
1043臺銀民權      -21000
1045臺銀新竹        4000
104A臺銀臺中       52000
104C臺銀高雄       -9000
104D臺銀金山        4000
               ...  
9A97永豐新竹        1100
9A99永豐中壢       -2000
9A9A永豐羅東      -12000
9A9B永豐中和        3000
9A9C永豐員林        1000
9A9E永豐板橋     -196000
9A9G永豐天母           0
9A9H永豐新莊        5000
9A9J永豐板新           0
9A9K永豐三重       10000
9A9L永豐台中       -2000
9A9M永豐南投      -30000
9A9N永豐桃盛        1000
9A9Q永豐豐原       -2000
9A9S永豐南京       11000
9A9U永豐中正       -2000
9A9W永豐市政  

# 主力買賣超

In [9]:
# 買方top 15
total.nlargest(15)

券商
8440摩根大通      544000
9A00永豐金       523000
1650瑞銀        448772
9800元大        391150
5920元    富    391000
8890大和國泰      300000
1160日    盛    241000
8840玉    山    201000
592I元富嘉義      175000
7750北    城    114000
989C元大板橋      107000
9825元大新營       81000
9309華南古亭       80000
779Z國票安和       77218
592l元富大裕       70000
dtype: int32

In [10]:
# 賣方 top 15
total.nsmallest(15)

券商
1470台灣摩根     -1205000
9869元大新盛      -539000
1440美林        -530000
1520瑞士信貸      -461000
884B玉山台中      -352000
700a兆豐忠孝      -328000
8960上海匯豐      -300000
700S兆豐大同      -274000
9211凱基台南      -273000
5851統一高雄      -200000
9A9E永豐板橋      -196000
5850統    一    -166000
1480美商高盛      -141000
9268凱基台北      -115000
9600富邦        -111000
dtype: int32

In [11]:
# 主力買賣超
total.nlargest(15).sum() + total.nsmallest(15).sum()

-1446860

# 買賣家數差

In [12]:
# 買賣家數差
(total > 0).sum() - (total < 0).sum()

294